# Evaluating Calliope (v 0.1.2)
This notebook allows us to evaluate Calliope and output relevant metrics for documentation.

In [5]:
import string
import random
import nltk
import pickle5 as pickle
import pandas as pd

df = pd.read_csv('../data/combined_ms.csv')
data = df.to_numpy()

word_string = ''
documents = []

for point in data:
    curr_tuple = (point[0].translate(str.maketrans('', '', string.punctuation)).split(), point[1])
    documents.append(curr_tuple)
    word_string = word_string + point[0].translate(str.maketrans('', '', string.punctuation))

random.shuffle(documents)
# print(documents[0])

all_words = nltk.FreqDist(w.lower() for w in word_string.split())

word_features = list(all_words)[:5]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

# Load pickled model:
calliope = pickle.load(open('../calliope.pickle', 'rb'))

In [6]:
# Ask for text input to classify:
test_banner = input("Input browser banner text: ")

print("'"+ test_banner + "' is " + calliope.classify(document_features(test_banner.split())))

'hello' is BAD


## Validation
Testing using data that was part of the training data set.

In [26]:
# Creating validation set
print("Validation set:")
pos_samples = [(content, label) for (content, label) in documents if label == "GOOD"]
neg_samples = [(content, label) for (content, label) in documents if label == "BAD"]
val_samples = random.sample(pos_samples, 10) + random.sample(neg_samples, 10)
print(*val_samples, sep = "\n")

Validation set:
(['This', 'site', 'uses', 'cookies', 'Some', 'of', 'these', 'cookies', 'are', 'essential', 'while', 'others', 'help', 'us', 'to', 'improve', 'your', 'experience', 'by', 'providing', 'insights', 'into', 'how', 'the', 'site', 'is', 'being', 'used', 'For', 'more', 'detailed', 'information', 'please', 'read', 'our', 'cookie', 'policy', 'httpstheisleofthanetnewscom', 'cookiepolicy', 'Necessary', 'Cookies', 'Necessary', 'cookies', 'enable', 'core', 'functionality', 'The', 'website', 'cannot', 'function', 'properly', 'without', 'these', 'cookies', 'and', 'can', 'only', 'be', 'disabled', 'by', 'changing', 'your', 'browser', 'preferences', 'Marketing', 'Cookies', 'Off', 'Marketing', 'cookies', 'are', 'used', 'to', 'track', 'visitors', 'across', 'websites', 'The', 'intention', 'is', 'to', 'display', 'ads', 'that', 'are', 'relevant', 'and', 'engaging', 'for', 'the', 'individual', 'user', 'and', 'thereby', 'more', 'valuable', 'for', 'publishers', 'and', 'third', 'part', 'advertiser

In [42]:
# Testing with validation set

from nltk.metrics import ConfusionMatrix
from nltk.metrics.scores import(precision, recall, f_measure)

val_set = [(document_features(content), label) for (content, label) in val_samples]

print("ACCURACY (Validation):",(nltk.classify.accuracy(calliope, val_set))*100)

print("\nCONFUSION MATRIX (Validation):")
print(ConfusionMatrix([label for (content, label) in val_set], [calliope.classify(content) for (content, label) in val_set]))

ACCURACY (Validation): 85.0

CONFUSION MATRIX (Validation):
     |   G |
     | B O |
     | A O |
     | D D |
-----+-----+
 BAD |<8>2 |
GOOD | 1<9>|
-----+-----+
(row = reference; col = test)



In [65]:
import pandas as pd
df_val = pd.DataFrame(val_samples)
df_val.rename(columns={0: 'Document', 1: 'Label'}, inplace=True)
df_val['Prediction'] = [calliope.classify(content) for (content, label) in val_set]

print(df_val)
df_val.to_csv('validation.csv', index=False)

                                             Document Label Prediction
0   [This, site, uses, cookies, Some, of, these, c...  GOOD       GOOD
1   [We, value, your, privacy, We, and, our, partn...  GOOD       GOOD
2   [We, and, our, partners, use, cookies, on, thi...  GOOD       GOOD
3   [Motor, Sport, uses, cookies, in, order, to, d...  GOOD       GOOD
4   [Dear, User, On, May, 25, 2018, Regulation, EU...  GOOD       GOOD
5   [We, value, your, privacy, We, and, our, partn...  GOOD       GOOD
6   [Cookie, consent, This, website, uses, cookies...  GOOD        BAD
7   [We, use, necessary, cookies, to, make, our, s...  GOOD       GOOD
8   [Cookies, usage, PRISA, uses, its, own, and, t...  GOOD       GOOD
9   [Accept, the, updated, privacy, cookie, policy...  GOOD       GOOD
10  [Accept, Cookies, Privacy, Policy, We, use, co...   BAD        BAD
11  [By, continuing, to, use, the, site, you, agre...   BAD        BAD
12  [By, using, this, site, you, agree, to, our, P...   BAD        BAD
13  [K

## Testing

Testing using data not from the training data set.

In [71]:
# Creating test set
import string

test_samples = [("Rappler uses cookies to customize user experience. By continued use, you agree to our privacy policy and accept our use of such cookies.", "GOOD"),
("We use cookies to ensure you get the best browsing experience. By continued use, you agree to our privacy policy and accept our use of such cookies. For further information, find out more.", "GOOD"),
("We use cookies to ensure you get the best experience on our website. By continuing, you are agreeing to our use of cookies. To find out more, please click this link.", "GOOD"),
("This site uses cookies. By continuing to browse the site, you are agreeing to our use of cookies.", "BAD"),
("This website uses cookies. By continuing to browse the website, you are agreeing to our use of cookies. Read More.", "BAD"),
("Welcome, Kapamilya! We use cookies to improve your browsing experience. Continuing to use this site means you agree to our use of cookies. Tell me more!", "GOOD"),
("We use cookies to ensure you get the best experience on PEP.ph. By continued use, you agree to our privacy policy and accept our use of such cookies. Find out more here.", "GOOD"),
("We use cookies to ensure we give you the best experience on our website.", "BAD"),
("Just so you know, we share cookies BeautyMNl uses cookies to give you the best shopping experience. Read on for our Privacy Policy", "BAD"),
("The IKEA website uses cookies, which make the site simpler to use. Find out more about browser cookies.", "BAD")]

word_string = ''
test_docs = []

for point in test_samples:
    curr_tuple = (point[0].translate(str.maketrans('', '', string.punctuation)).split(), point[1])
    test_docs.append(curr_tuple)
    word_string = word_string + point[0].translate(str.maketrans('', '', string.punctuation))

print("Testing set:")
print(*test_docs, sep = "\n")

Testing set:
(['Rappler', 'uses', 'cookies', 'to', 'customize', 'user', 'experience', 'By', 'continued', 'use', 'you', 'agree', 'to', 'our', 'privacy', 'policy', 'and', 'accept', 'our', 'use', 'of', 'such', 'cookies'], 'GOOD')
(['We', 'use', 'cookies', 'to', 'ensure', 'you', 'get', 'the', 'best', 'browsing', 'experience', 'By', 'continued', 'use', 'you', 'agree', 'to', 'our', 'privacy', 'policy', 'and', 'accept', 'our', 'use', 'of', 'such', 'cookies', 'For', 'further', 'information', 'find', 'out', 'more'], 'GOOD')
(['We', 'use', 'cookies', 'to', 'ensure', 'you', 'get', 'the', 'best', 'experience', 'on', 'our', 'website', 'By', 'continuing', 'you', 'are', 'agreeing', 'to', 'our', 'use', 'of', 'cookies', 'To', 'find', 'out', 'more', 'please', 'click', 'this', 'link'], 'GOOD')
(['This', 'site', 'uses', 'cookies', 'By', 'continuing', 'to', 'browse', 'the', 'site', 'you', 'are', 'agreeing', 'to', 'our', 'use', 'of', 'cookies'], 'BAD')
(['This', 'website', 'uses', 'cookies', 'By', 'continui

In [72]:
# Testing with test set

from nltk.metrics import ConfusionMatrix

test_set = [(document_features(d), c) for (d, c) in test_docs]

print("ACCURACY (Testing):",(nltk.classify.accuracy(calliope, test_set))*100)

print("\nCONFUSION MATRIX (Testing):")
print(ConfusionMatrix([label for (content, label) in test_set], [calliope.classify(content) for (content, label) in test_set]))

ACCURACY (Testing): 80.0

CONFUSION MATRIX (Testing):
     |   G |
     | B O |
     | A O |
     | D D |
-----+-----+
 BAD |<5>. |
GOOD | 2<3>|
-----+-----+
(row = reference; col = test)



In [74]:
import pandas as pd
df_test = pd.DataFrame(test_samples)
df_test.rename(columns={0: 'Document', 1: 'Label'}, inplace=True)
df_test['Prediction'] = [calliope.classify(content) for (content, label) in test_set]

print(df_test)
df_test.to_csv('test.csv', index=False)

                                            Document Label Prediction
0  Rappler uses cookies to customize user experie...  GOOD       GOOD
1  We use cookies to ensure you get the best brow...  GOOD       GOOD
2  We use cookies to ensure you get the best expe...  GOOD        BAD
3  This site uses cookies. By continuing to brows...   BAD        BAD
4  This website uses cookies. By continuing to br...   BAD        BAD
5  Welcome, Kapamilya! We use cookies to improve ...  GOOD        BAD
6  We use cookies to ensure you get the best expe...  GOOD       GOOD
7  We use cookies to ensure we give you the best ...   BAD        BAD
8  Just so you know, we share cookies BeautyMNl u...   BAD        BAD
9  The IKEA website uses cookies, which make the ...   BAD        BAD
